In [802]:
# TODO: karar matrisi kıyaslama ekle
# TODO: her indiktaör için ayrı request yapısı ekle
# TODO: performans metrikleri ekle
# TODO: her strateji için ayrı sınıf yapısı ekle
# TODO: daha modüler hale getir

In [803]:
import pandas as pd
symbol = "GOOGL"
date = "2024-07-10"

In [804]:
internal_google = pd.read_csv(f'C:\\Users\\Özlem Nur\\Desktop\\tradebot_backend\\tradebot_ai\\reports\\internal\\{symbol}_internal_report.csv')
external_google = pd.read_csv(f'C:\\Users\\Özlem Nur\\Desktop\\tradebot_backend\\tradebot_ai\\reports\\external\\{symbol}_external_report.csv')

In [805]:
internal_google.head()

,id,symbol,indicator,signal,date
0,0,GOOGL,rsi,Hold,2015-01-04
1,1,GOOGL,rsi,Hold,2015-01-11
2,2,GOOGL,rsi,Hold,2015-01-18
3,3,GOOGL,rsi,Hold,2015-01-25
4,4,GOOGL,rsi,Hold,2015-02-01


In [806]:
external_google.head()

,id,symbol,indicator,signal,date
0,1,GOOGL,rsi,Hold,2015-01-04
1,2,GOOGL,macd,Buy,2015-01-04
2,3,GOOGL,bollinger,Hold,2015-01-04
3,4,GOOGL,ma-cross,Sell,2015-01-04
4,5,GOOGL,dmi,Sell,2015-01-04


In [807]:
INDICATORS = ["rsi", "macd", "bollinger", "ma-cross", "dmi", "mfi", "trendline"]

In [808]:
internal_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899 entries, 0 to 3898
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3899 non-null   int64 
 1   symbol     3899 non-null   object
 2   indicator  3899 non-null   object
 3   signal     3899 non-null   object
 4   date       3899 non-null   object
dtypes: int64(1), object(4)
memory usage: 152.4+ KB


In [809]:
internal_google.groupby('indicator')["signal"].unique()

indicator
bollinger    [Hold, Sell, Buy]
dmi                [Buy, Sell]
ma-cross                 [Buy]
macd               [Buy, Sell]
mfi          [Hold, Sell, Buy]
rsi          [Hold, Sell, Buy]
trendline    [Hold, Sell, Buy]
Name: signal, dtype: object

In [810]:
internal_google.join(external_google.set_index(['date', 'indicator']), on=['date', 'indicator'], lsuffix='_internal', rsuffix='_external')
merged = pd.merge(internal_google, external_google, on=['date', 'indicator'], suffixes=('_internal', '_external'))
merged.head()

,id_internal,symbol_internal,indicator,signal_internal,date,id_external,symbol_external,signal_external
0,0,GOOGL,rsi,Hold,2015-01-04,1,GOOGL,Hold
1,1,GOOGL,rsi,Hold,2015-01-11,8,GOOGL,Hold
2,2,GOOGL,rsi,Hold,2015-01-18,15,GOOGL,Hold
3,3,GOOGL,rsi,Hold,2015-01-25,22,GOOGL,Hold
4,4,GOOGL,rsi,Hold,2015-02-01,29,GOOGL,Hold


In [811]:
merged_results = merged.groupby('indicator').apply(lambda x: (x['signal_internal'] == x['signal_external']).mean() * 100)
merged_results = merged_results.reset_index()
merged_results.columns = ['indicator', 'accuracy_percentage']
# for testing purpose
#merged_results = merged_results.drop(index=1, axis=0)
# -------------------
merged_results["accuracy_percentage"] = merged_results["accuracy_percentage"] / len(INDICATORS)
merged_results

C:\Users\Özlem Nur\AppData\Local\Temp\ipykernel_19488\2102003345.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_results = merged.groupby('indicator').apply(lambda x: (x['signal_internal'] == x['signal_external']).mean() * 100)


,indicator,accuracy_percentage
0,bollinger,14.285714
1,dmi,0.230828
2,ma-cross,10.848936
3,macd,14.003591
4,mfi,14.260067
5,rsi,13.952295
6,trendline,0.615542


In [812]:
import requests

url = f"http://localhost:8082/ta/getSignals/{symbol}?date={date}T00:00:00"

response = requests.get(url)
data = response.json()
data

{'ma-cross': 1,
 'mfi': 0,
 'trendline': 0,
 'rsi': -1,
 'macd': 1,
 'dmi': -1,
 'bollinger': 0}

In [813]:
df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['indicator', 'score']
df["score"] = df["score"]
df

,indicator,score
0,ma-cross,1
1,mfi,0
2,trendline,0
3,rsi,-1
4,macd,1
5,dmi,-1
6,bollinger,0


In [814]:
merged_indicators = df.join(merged_results.set_index('indicator'), on='indicator')
merged_indicators

,indicator,score,accuracy_percentage
0,ma-cross,1,10.848936
1,mfi,0,14.260067
2,trendline,0,0.615542
3,rsi,-1,13.952295
4,macd,1,14.003591
5,dmi,-1,0.230828
6,bollinger,0,14.285714


In [815]:
# for testing purpose
#merged_indicators = merged_indicators.drop(index=5, axis=0)
#if (df["indicator"] == "trendline").any():
#    df.loc[df["indicator"] == "trendline", "score"] *= -1
# -------------------

In [816]:
merged_indicators

,indicator,score,accuracy_percentage
0,ma-cross,1,10.848936
1,mfi,0,14.260067
2,trendline,0,0.615542
3,rsi,-1,13.952295
4,macd,1,14.003591
5,dmi,-1,0.230828
6,bollinger,0,14.285714


In [817]:
if (merged_indicators["indicator"] == "macd").any():
    merged_indicators.loc[merged_indicators["indicator"] == "macd", "score"] *= -1
if (merged_indicators["indicator"] == "ma-cross").any():
    merged_indicators.loc[merged_indicators["indicator"] == "ma-cross", "score"] *= -1

In [818]:
merged_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   indicator            7 non-null      object 
 1   score                7 non-null      int64  
 2   accuracy_percentage  7 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 300.0+ bytes


In [819]:
karar_matrisi = merged_indicators.copy()
karar_matrisi[["Katman", "Sinyal/Skor", "Güven Ağırlığı"]] = karar_matrisi[["indicator", "score", "accuracy_percentage"]]
karar_matrisi = karar_matrisi.drop(columns=["indicator", "score", "accuracy_percentage"])
karar_matrisi["Sinyal/Skor"] = karar_matrisi["Sinyal/Skor"].astype("object")
karar_matrisi["Sinyal/Skor"] = karar_matrisi["Sinyal/Skor"].map(lambda x: "BUY" if x == 1 else "SELL" if x == -1 else "HOLD")
karar_matrisi["Güven Ağırlığı"] = round(karar_matrisi["Güven Ağırlığı"], 2)
display(karar_matrisi)

,Katman,Sinyal/Skor,Güven Ağırlığı
0,ma-cross,SELL,10.85
1,mfi,HOLD,14.26
2,trendline,HOLD,0.62
3,rsi,SELL,13.95
4,macd,SELL,14.00
5,dmi,SELL,0.23
6,bollinger,HOLD,14.29


In [820]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
minmax_scaled_features = minmax_scaler.fit_transform(merged_indicators["accuracy_percentage"].values.reshape(-1, 1))
minmax_scaled_features

array([[0.75547445],
       [0.99817518],
       [0.02737226],
       [0.97627737],
       [0.97992701],
       [0.        ],
       [1.        ]])

In [821]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler(with_mean=True, with_std=True)
standard_scaled_features = standard_scaler.fit_transform(merged_indicators["accuracy_percentage"].values.reshape(-1, 1))
standard_scaled_features

array([[ 0.18444558],
       [ 0.753051  ],
       [-1.52137067],
       [ 0.70174825],
       [ 0.71029871],
       [-1.5854991 ],
       [ 0.75732623]])

In [822]:
merged_indicators["accuracy_percentage"] = standard_scaled_features

In [823]:
merged_indicators['final_score'] = merged_indicators['score'] * (merged_indicators['accuracy_percentage'] / 100)
merged_indicators[["indicator", "final_score"]]

,indicator,final_score
0,ma-cross,-0.001844
1,mfi,0.000000
2,trendline,-0.000000
3,rsi,-0.007017
4,macd,-0.007103
5,dmi,0.015855
6,bollinger,0.000000


In [824]:
total_net_score = merged_indicators["final_score"].sum()
total_net_score

np.float64(-0.00010993445246210501)

In [835]:
sell, hold, buy = -1, 0, 1

def net_score(total_net_score):
    if total_net_score <= -1.0 and total_net_score < -0.3:
        decision = "SELL"
    elif total_net_score >= -0.3 and total_net_score < 0.2:
        decision = "HOLD"
    elif total_net_score >= 0.2 and total_net_score <= 1.0:
        decision = "BUY"

    return decision

print(f"Symbol {symbol} on {date}")
print(f"Total net score: {total_net_score}")
print(f"Decision: {net_score(total_net_score)}")

Symbol GOOGL on 2024-07-10
Total net score: -0.00010993445246210501
Decision: HOLD


In [826]:
import os
import IPython

os.environ["SSL_CERT_FILE"] = r"C:\\Users\\Public\\cacert.pem"
os.environ["REQUESTS_CA_BUNDLE"] = r"C:\\Users\\Public\\cacert.pem"

from TAResults import get_signal
yf_result = get_signal(symbol, date)
IPython.display.clear_output()

In [827]:
yf_result

,symbol,indicator,signal,date
0,GOOGL,rsi,Sell,2024-07-10
1,GOOGL,macd,Buy,2024-07-10
2,GOOGL,bollinger,Hold,2024-07-10
3,GOOGL,ma-cross,Buy,2024-07-10
4,GOOGL,dmi,Buy,2024-07-10
5,GOOGL,mfi,Hold,2024-07-10
6,GOOGL,trendline,Buy,2024-07-10


In [828]:
yf_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   symbol     7 non-null      object
 1   indicator  7 non-null      object
 2   signal     7 non-null      object
 3   date       7 non-null      object
dtypes: object(4)
memory usage: 356.0+ bytes


In [829]:
yf_result["signal"] = yf_result["signal"].map(lambda x: "1" if x == "Buy" else "-1" if x == "Sell" else "0")
yf_result["signal"] = yf_result["signal"].astype("int32")
yf_result[["Katman", "Sinyal/Skor"]] = yf_result[["indicator", "signal"]]
yf_result = yf_result.drop(columns=["symbol", "date", "indicator", "signal"])
yf_result

,Katman,Sinyal/Skor
0,rsi,-1
1,macd,1
2,bollinger,0
3,ma-cross,1
4,dmi,1
5,mfi,0
6,trendline,1


In [830]:
karar_matrisi["Sinyal/Skor"] = karar_matrisi["Sinyal/Skor"].map(lambda x: "1" if x == "BUY" else "-1" if x == "SELL" else "0")
karar_matrisi["Sinyal/Skor"] = karar_matrisi["Sinyal/Skor"].astype("int32")
karar_matrisi_latest = karar_matrisi.merge(yf_result, on="Katman", how="outer", suffixes=('_internal', '_yfinance'))

In [831]:
karar_matrisi_latest[["Katman", "Sinyal/Skor_internal", "Sinyal/Skor_yfinance", "Güven Ağırlığı"]]

,Katman,Sinyal/Skor_internal,Sinyal/Skor_yfinance,Güven Ağırlığı
0,bollinger,0,0,14.29
1,dmi,-1,1,0.23
2,ma-cross,-1,1,10.85
3,macd,-1,1,14.00
4,mfi,0,0,14.26
5,rsi,-1,-1,13.95
6,trendline,0,1,0.62


In [834]:
yf_total_net_score = karar_matrisi_latest["Sinyal/Skor_yfinance"].dot(karar_matrisi_latest["Güven Ağırlığı"]) /100
yf_total_net_score

np.float64(0.11749999999999998)

In [838]:
print(net_score(yf_total_net_score))

HOLD
